<a href="https://colab.research.google.com/github/Vik-Gt-321/RL-AI/blob/main/Propositional_Logic_to_CNF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Notes**

All functions are optimized to take in Propositions with a max of two operands i.e binary

Print function can be optimized for more than binary cases, although not necessary

# Class Definitions and Testing 

In [ ]:
class Proposition:
    def __init__(self, op, *operands):
        self.op = op
        self.operands = operands
    
    def __eq__(self, other):
        if isinstance(other, Proposition):
            return self.op == other.op and self.operands == other.operands
        return False
    
    def __hash__(self):
        return hash((self.op, self.operands))
    
    def __str__(self):
        if self.is_variable():
            # print(self.op, "var")
            return self.op
        elif self.is_unary():
            # print(self.op)
            # print(self.operands[0], "unary")
            return f"({self.op}{self.operands[0]})"
        else:
            # print(self.op, "prop")
            # for op in self.operands:
            #   print(op)
            
            operands_str = ""

            operands_str += (str(self.operands[0]))
            operands_str += " "
            operands_str += (str(self.op))
            operands_str += " "
            operands_str += (str(self.operands[1]))
            # operands_str = " ".join(str(operand) for operand in self.operands)
            return f"({operands_str})"
    
    def is_variable(self):
      return isinstance(self.op, str) and not self.operands
    
    def is_unary(self):
        return self.op in {"~"} and len(self.operands) == 1
    
    def is_binary(self):
        return self.op in {"&", "|", "->"} and len(self.operands) == 2
    
    def is_negation(self):
        return self.op == "~" and self.operands[0].is_variable()
    
    def is_conjunction(self):
        return self.op == "&" and all(op.is_proposition() for op in self.operands)
    
    def is_disjunction(self):
        return self.op == "|" and all(op.is_proposition() for op in self.operands)
    
    def is_implication(self):
        return self.op == "->" and all(op.is_proposition() for op in self.operands)
    
    def is_proposition(self):
        return self.is_variable() or self.is_unary() or self.is_binary()

    def print_proposition(self):
        print(str(self))

    def eliminate_implications(self):
      if self.is_variable():
          # print(f"var elim of " ,{self.op})

          return self

      elif self.is_unary():
          # print("unary elimin")

          new_operand = self.operands[0].eliminate_implications()
          return Proposition(self.op, new_operand)
      else:
          # print("binary elim")
          # left, right = [op.eliminate_implications() for op in self.operands]
          # print(str(self.operands[0]))

          left, right = self.operands[0], self.operands[1]
          left = left.eliminate_implications()
          right = right.eliminate_implications()
          
          # right.print_proposition()
          if self.is_implication():
              # print("is impl if")
              # print(left)
              # print(isinstance(left, Proposition))
              # print(right, "right")
              new_proposition = Proposition("|", Proposition("~", left), right)
              return new_proposition   
          
          # right.print_proposition()
          # Proposition(self.op, left, right).print_proposition()
          return Proposition(self.op, left, right)

    def negate(self):
      '''
      returns the negated Propostion object
      '''

      # print("inside negate function for ", self.print_proposition())
      if self.op != "~":
        # print("self op is not ~")
        if self.is_variable():
          return Proposition("~", self)
        if self.is_unary():
          # print("inside unary")
          other_than_operator = self.operands[0]
          if other_than_operator.is_variable():
            return other_than_operator.negate()

          if other_than_operator.is_unary():
            if other_than_operator.op == "~":
              return other_than_operator
            else:
              return other_than_operator.operands[0].negate()

        if self.is_binary():
          self = self.eliminate_implications()
          left, right = self.operands
          # print("inside move neg inwards", left, right)
          if self.is_conjunction():
              # print("inside conjucntion")
              return Proposition("|", left.negate(), right.negate())
          if self.is_disjunction():
              # print("inside dis")
              return Proposition("&", left.negate(), right.negate())
      else:
        return self.operands[0]

    def move_negation_inwards(self):
        '''
        moves the negative inwards and returns the Negative Normal Form
        '''
        if self.is_variable():
            # print("inside var")
            return self

        if self.is_unary():
            # print("inside unary", self.print_proposition())
            
            other_than_neg = self.operands[0]
            # print("other than neg", other_than_neg)
            if self.op == "~":
              return other_than_neg.negate()
            
            return other_than_neg.move_negation_inwards()
      

        if self.is_binary():
            # print("inside binary")
            self = self.eliminate_implications()
            self.print_proposition()
            left, right = self.operands
            # print("inside move neg inwards is binary", left, right)
            if self.is_conjunction():
                # print("inside conjucntion")
                return Proposition("&", left.move_negation_inwards(), right.move_negation_inwards())
            if self.is_disjunction():
                # print("inside dis")
                return Proposition("|", left.move_negation_inwards(), right.move_negation_inwards())

        return self

# Helper functions (Some defined outside class, others inside)


*   `distribute_disjunctions(proposition)`
*   `move_negation_inwards(self)`
*   `negate(self)`
*   `eliminate_implications(self)`






### Move negation inwards testing

In [ ]:
p = Proposition("~", Proposition("&", Proposition("p"), Proposition("~", Proposition("q"))))
q = Proposition("~", Proposition("|", Proposition("~", Proposition("&", Proposition("p"), Proposition("q"))),
                                 Proposition("r")))
r = Proposition("&", Proposition("~", Proposition("|", Proposition("p"), Proposition("~", Proposition("q")))), Proposition("r"))

q.move_negation_inwards().print_proposition()

((p & q) & (~r))


### Eliminate implications testing 

In [ ]:
p1 = Proposition("->", Proposition("p"), Proposition("q"))
p2 = Proposition("&", Proposition("r"), p1)

# Before elimination
print(p2)  

# After elimination
p2 = p2.eliminate_implications()
print(p2)  


(r & (p -> q))
(r & ((~p) | q))


In [ ]:
p = Proposition("->", Proposition("p"), Proposition("q"))
p = p.eliminate_implications() 
p.print_proposition()

((~p) | q)


### Negate testing 

In [ ]:
q = Proposition("~", Proposition("q"))
r = Proposition("r")
p1 = Proposition("->", Proposition("p"), Proposition("q"))
p =  Proposition("&", Proposition("P"), Proposition("|", Proposition("Q"), Proposition("~", Proposition("R"))))

conjunction = Proposition("|", q, r)
con = p.negate()
con.print_proposition()

((~P) | ((~Q) & R))


In [ ]:
def distribute_disjunctions(proposition):
  '''
  distributes ors over ands
  '''
  if proposition.is_disjunction():
      left, right = proposition.operands
      if left.is_conjunction():
          left1, left2 = left.operands
          new_proposition = Proposition("&", Proposition("|", left1, right), Proposition("|", left2, right))
          return distribute_disjunctions(new_proposition)
      elif right.is_conjunction():
          right1, right2 = right.operands
          new_proposition = Proposition("&", Proposition("|", left, right1), Proposition("|", left, right2))
          return distribute_disjunctions(new_proposition)
      return proposition
  elif proposition.is_conjunction():
      new_operands = [distribute_disjunctions(operand) for operand in proposition.operands]
      return Proposition("&", *new_operands)
  return proposition

In [ ]:
p = Proposition("|", Proposition("&", Proposition("p"), Proposition("q")), Proposition("&", Proposition("r"), Proposition("s")))
distribute_disjunctions(p).print_proposition()

(((p | r) & (p | s)) & ((q | r) & (q | s)))


# Proposition To CNF Algorithm

In [ ]:
def to_cnf(proposition):
  '''
  pipeline for converting Propostion object to its Conjunctive normal form
  '''
  proposition = proposition.eliminate_implications()
  proposition = proposition.move_negation_inwards()
  proposition = distribute_disjunctions(proposition)
  return proposition

In [ ]:
p = Proposition("~", Proposition("&", Proposition("P"), Proposition("Q")))
to_cnf(p).print_proposition()

((~P) | (~Q))


In [ ]:
p = Proposition("->", Proposition("a"), Proposition("&", Proposition("b"), Proposition("c")))
to_cnf(p).print_proposition()

((~a) | (b & c))
(b & c)
(((~a) | b) & ((~a) | c))


In [ ]:
p = Proposition("&",
    Proposition("->", Proposition("P"), Proposition("Q")),
    Proposition("~", Proposition("|", Proposition("P"), 
                                 Proposition("&", Proposition("~", Proposition("Q")), Proposition("R"))))
    # Proposition("|", Proposition("S"), Proposition("~", Proposition("P")))
)

to_cnf(p).print_proposition()

(((~P) | Q) & (~(P | ((~Q) & R))))
((~P) | Q)
(((~P) | Q) & ((~P) & (Q | (~R))))


##### Problem 1:If John is a bachelor, then John is unmarried.
John is a bachelor.
Therefore, John is unmarried.


In [ ]:
p1a = Proposition("&",Proposition("->",Proposition("B"),Proposition("U")),Proposition("B"))
p1 = Proposition("&", p1a , Proposition("U"))
to_cnf(p1).print_proposition()

((((~B) | U) & B) & U)
(((~B) | U) & B)
((~B) | U)
((((~B) | U) & B) & U)


#### Problem 2:

In [ ]:
#(P → Q) ∧ (R → S) ∧ (P ∨ R) ∧ ¬(Q ∧ S)

p2a = Proposition("->",Proposition("P"),Proposition("Q"))
p2b = Proposition("->",Proposition("R"),Proposition("S"))
p2c = Proposition("|",Proposition("P"),Proposition("R"))
p2d = Proposition("~", Proposition("&", Proposition("Q"), Proposition("S")))

p21 = Proposition("&",p2a,p2b)
p22 = Proposition("&",p2c,p2d)

p2 = Proposition("&",p21,p22)

to_cnf(p2).print_proposition()

((((~P) | Q) & ((~R) | S)) & ((P | R) & (~(Q & S))))
(((~P) | Q) & ((~R) | S))
((~P) | Q)
((~R) | S)
((P | R) & (~(Q & S)))
(P | R)
((((~P) | Q) & ((~R) | S)) & ((P | R) & ((~Q) | (~S))))


In [ ]:
# p3a = Proposition("&", Proposition("P"),Proposition("Q"))
# p3b = Proposition("|", Proposition("R"), Proposition("S"))

# p3 = Proposition("->", p3a, p3b)

# to_cnf(p3).print_proposition()